<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#获取基金费率等信息" data-toc-modified-id="获取基金费率等信息-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>获取基金费率等信息</a></span></li><li><span><a href="#获取基金经理信息" data-toc-modified-id="获取基金经理信息-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>获取基金经理信息</a></span></li><li><span><a href="#获取基金规模，年度收益信息" data-toc-modified-id="获取基金规模，年度收益信息-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>获取基金规模，年度收益信息</a></span></li><li><span><a href="#获取多只基金的信息" data-toc-modified-id="获取多只基金的信息-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>获取多只基金的信息</a></span></li><li><span><a href="#相关阅读" data-toc-modified-id="相关阅读-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>相关阅读</a></span></li></ul></div>

In [1]:
import pandas as pd
import datetime
import numpy as np
import tushare as ts
import os
import time

print(f'tushare version: {ts.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')

tushare version: 1.2.84
pandas version: 1.4.1
numpy version: 1.22.3


使用tushare 获取财经数据，首先需要注册，注册地址如下：

[点击注册tushare](https://tushare.pro/register?reg=129033)

然后按照下面的代码设置 token

以上方法只需要在第一次或者token失效后调用，完成调取tushare数据凭证的设置，正常情况下不需要重复设置。

In [2]:
# token='XXXXXXX'  
# ts.set_token(token)

pro = ts.pro_api()

## 获取基金费率等信息

In [3]:
# 获取基金中文名称信息
df_e = pro.fund_basic(market='E')
df_o = pro.fund_basic(market='O')
# df_fund_info = df_e.append(df_o)
df_fund_info = pd.concat([df_e,df_o])
df_fund_info = df_fund_info[['ts_code','name', 'management','m_fee', 'c_fee','found_date']]
df_fund_info.columns = ['fund_code','fund_name', 'management','m_fee', 'c_fee','found_date']
df_fund_info = df_fund_info.reset_index(drop=True)
df_fund_info

,fund_code,fund_name,management,m_fee,c_fee,found_date
0,508018.SH,华夏中国交建REIT,华夏基金,0.10,0.01,20220413
1,159618.SZ,光伏ETF华安,华安基金,0.50,0.10,20220408
2,501217.SH,行业配置FOF,华夏基金,0.80,0.20,20220408
3,513530.SH,港股通红利ETF,华泰柏瑞基金,0.50,0.10,20220408
4,517760.SH,消费TOPETF,浦银安盛基金,0.50,0.10,20220401
...,...,...,...,...,...,...
16625,000198.OF,天弘余额宝,天弘基金,0.30,0.08,20130529
16626,4710071.OF,汇添富理财7天A,汇添富基金,0.27,0.08,20130529
16627,000117.OF,广发轮动配置,广发基金,1.50,0.25,20130528
16628,000082.OF,嘉实研究阿尔法,嘉实基金,1.50,0.25,20130528


## 获取基金经理信息

In [4]:
# 获取单个基金的信息

def get_fund(fund_code,df_fund_info):    
    # 获取基金经理信息，以及开始管理该基金的日期
    df_manager = pro.fund_manager(ts_code=fund_code)
    df_manager = df_manager[df_manager['end_date'].isna()]
    df_manager = df_manager.sort_values('begin_date',ascending=True).head(1)
    begin_date = df_manager['begin_date'].values[0] # 开始管理该基金的日期
    df_manager = df_manager[['ts_code', 'name', 'gender','begin_date']]
    df_manager.columns = ['fund_code', 'manager_name', 'gender','begin_date']
    
    # 合并
    df_fund = pd.merge(df_manager,df_fund_info,how='left',on='fund_code')

    return df_fund

In [5]:
# 110011.OF,易方达优质精选
df_fund = get_fund('110011.OF',df_fund_info) 
df_fund

,fund_code,manager_name,gender,begin_date,fund_name,management,m_fee,c_fee,found_date
0,110011.OF,张坤,M,20210910,易方达优质精选,易方达基金,1.5,0.25,20210910


## 获取基金规模，年度收益信息

In [6]:
# 获取单支基金的年度收益,基金规模等信息

def get_returns(fund_code,start_year):
    """获取单支基金的年度收益,基金规模等信息
    Keyword arguments:
    fund_code -- 基金代码，如：110011.OF
    start_year -- 开始获取基金数据的年份
    """
    df_fund_nav = pro.fund_nav(ts_code=fund_code)
    df_fund_nav['date'] = pd.to_datetime(df_fund_nav['nav_date'])
    df_fund_nav = df_fund_nav.set_index('date').sort_index()
    
    years = df_fund_nav['accum_nav'].resample('AS').sum().to_period('A')
    # 获取年度数据，类型为 pandas 的 period
    years = years.index.tolist()
    
    # 有些基金的开始日期晚于2018年，需要对齐列，补空白
    year_list = [yr.year for yr in years if yr.year>=start_year]
    
    # 将 period 时间数据转为 string 的年度数据
    years = [str(yr.year) for yr in years]

    df_yrs_index = pd.DataFrame()
    for yr in years:
        df_yr_index = df_fund_nav.loc[yr].tail(1)
#         df_yrs_index = df_yrs_index.append(df_yr_index) # append 将被替代
        df_yrs_index = pd.concat([df_yrs_index,df_yr_index])
#     df_yrs = df_fund_nav.head(1).append(df_yrs_index)
    df_yrs = pd.concat([df_fund_nav.head(1),df_yrs_index])

    # 计算每年的收益率，即涨跌幅度
    df_yrs['returns'] = df_yrs['accum_nav'].pct_change()

    # 删除收益率为 NA 的行 （第1天有数据记录的日期）
    df_yrs = df_yrs.dropna(subset=['returns'])

    # 筛选自开始年份以来的数据
    df_yrs = df_yrs.loc[str(start_year):]

    df_yrs = df_yrs.reset_index()
    df_yrs['year'] = df_yrs['date'].dt.year

    # 透视表
    df_yr_returns = pd.pivot_table(
        df_yrs, index=['ts_code'],
        values=['returns'], columns=['year'], fill_value=""
    )

    # 将多层索引转变为单层索引
    df_yr_returns = df_yr_returns['returns']
    df_yr_returns = df_yr_returns.reset_index()
    df_yr_returns.columns.name = None
    
    df_yr_returns = df_yr_returns.rename(columns={'ts_code':'fund_code'})
    
    # 基金份额
    df_fund_share = pro.fund_share(ts_code=fund_code).head(1)
    df_fund_share.columns = ['fund_code', 'fd_share_date', 'fd_share', 'fund_type', 'market']
    # fd_share,单位是 万份
    fd_share_date = df_fund_share['fd_share_date'].values[0] # 份额对应的日期
    
    # 份额日期的净值数据
    df_ann_nav = df_fund_nav.loc[fd_share_date:fd_share_date].sort_values('update_flag',ascending=False).head(1)
    df_ann_nav = df_ann_nav[['ts_code', 'accum_nav','unit_nav']]
    df_ann_nav.columns = ['fund_code', 'accum_nav','unit_nav']
    
    # 计算基金规模，amount，单位：亿元
    df_fund_amount = pd.merge(df_fund_share,df_ann_nav,how='left',on='fund_code')    
    df_fund_amount['amount'] = df_fund_amount['fd_share'] * df_fund_amount['unit_nav']/10000
    df_fund_amount = df_fund_amount[['fund_code','amount','fd_share_date']]
    
    # 合并数据    
    df_yr_returns = pd.merge(df_fund_amount,df_yr_returns,how='left',on='fund_code')

    for yr in year_list:
        if yr not in df_yr_returns.columns.tolist():
            df_yr_returns[yr]=np.nan
    
    return df_yr_returns

In [7]:
df_return = get_returns('110011.OF',2018)
df_return

,fund_code,amount,fd_share_date,2018,2019,2020,2021,2022
0,110011.OF,216.61701,20210930,-0.129849,0.585684,0.715145,-0.108154,-0.160545


## 获取多只基金的信息

In [8]:
# 获取多只基金的信息

def get_data_fund(df_fund_info,fund_code_short,start_year):
    df_filter_info = pd.DataFrame()
    for code in fund_code_short:
        df_tmp = df_fund_info[df_fund_info['fund_code'].str.contains(code)].head(1)
#         df_filter_info = df_filter_info.append(df_tmp)
        df_filter_info = pd.concat([df_filter_info,df_tmp])
    df_filter_info = df_filter_info.reset_index(drop=True)   
    fund_codes = df_filter_info['fund_code'].tolist()
    # fund_codes

#     df = pd.DataFrame()
    frames = [] # 用来装载 df_fund_merge
    i=0
    for code in fund_codes:
        print(f'i:{i},code:{code}')
        if i<78:
#             print(code)
#             print(f'i:{i},code:{code}')
            df_fund = get_fund(code,df_fund_info)    
            df_return = get_returns(code,start_year)
            df_fund_merge = pd.merge(df_fund,df_return,how='left',on='fund_code')
            frames.append(df_fund_merge)
            i +=1
        else:
            i=0
            time.sleep(61)
#         df = df.append(df_fund_merge)
    df = pd.concat(frames)
    df = df.reset_index(drop=True)
    
    df = df.rename(columns={
        'fund_code':'基金代码','manager_name':'基金经理',
        'gender':'性别','begin_date':'上任日期',
        'fund_name':'基金名称','management':'基金公司',
        'm_fee':'管理费','c_fee':'托管费',
        'found_date':'成立时间','amount':'基金规模(亿元)',
        'fd_share_date':'规模对应日期'
    })
    
    # 调整列的排序
    cols = df.columns.tolist()
    col_1 = cols[:4]
    col_2 = cols[4:5]
    col_3 = cols[5:]
    cols = col_2 + col_1 + col_3
    df = df[cols]

    return df

设置具体的参数

In [9]:
# 需要获取的基金代码简称列表
fund_code_short = ['001938', '162605', '005911', '005827','007119', 
                   '110011', '161005', '000800', '519736', '163402',
                   '003095', '163406', '002190', '164908', '000772']

# 设置基金收益开始的年份
start_year = 2018
# 设置基金收益截止的年份
end_year = 2022
year_list = list(range(start_year,end_year+1))

df_fund_final = get_data_fund(df_fund_info,fund_code_short,start_year)
df = df_fund_final.sort_values('基金经理',ascending=True,ignore_index=True)
# df.to_csv('./data/fund_manager.csv',index=False)
df

i:0,code:001938.OF
i:1,code:162605.SZ
i:2,code:005911.OF
i:3,code:005827.OF
i:4,code:007119.OF
i:5,code:110011.OF
i:6,code:161005.SZ
i:7,code:000800.OF
i:8,code:519736.OF
i:9,code:163402.SZ
i:10,code:003095.OF
i:11,code:163406.SZ
i:12,code:002190.OF
i:13,code:164908.OF
i:14,code:000772.OF


,基金名称,基金代码,基金经理,性别,上任日期,基金公司,管理费,托管费,成立时间,基金规模(亿元),规模对应日期,2018,2019,2020,2021,2022
0,睿远成长价值A,007119.OF,傅鹏博,M,20190326,睿远基金,1.5,0.15,20190326,326.890380,20211231,NaN,0.207500,0.709979,0.026104,-0.315099
1,中欧时代先锋A,001938.OF,刘伟伟,M,20211217,中欧基金,1.5,0.25,20151103,190.994106,20210930,-0.083308,0.442859,0.375462,0.072199,-0.177790
2,景顺鼎益LOF,162605.SZ,刘彦春,M,20150710,景顺长城基金,1.5,0.25,20050316,226.506554,20210930,-0.053947,0.195658,0.362284,-0.052353,-0.097272
3,广发双擎升级A,005911.OF,刘格菘,M,20181102,广发基金,1.5,0.25,20181102,141.091313,20210930,-0.017000,1.216887,0.643585,0.042466,-0.274600
4,易方达蓝筹精选,005827.OF,张坤,M,20180905,易方达基金,1.5,0.25,20180905,698.473170,20210930,-0.052900,0.551156,0.950922,-0.098950,-0.195160
5,易方达优质精选,110011.OF,张坤,M,20210910,易方达基金,1.5,0.25,20210910,216.617010,20210930,-0.129849,0.585684,0.715145,-0.108154,-0.160545
6,富国天惠LOF,161005.SZ,朱少醒,M,20051116,富国基金,1.5,0.25,20051116,367.843794,20210930,-0.113196,0.215553,0.256741,0.002546,-0.126853
7,华商未来主题,000800.OF,李双全,M,20190823,华商基金,1.5,0.25,20141014,5.101099,20210930,-0.373640,0.403475,0.221458,0.159910,-0.235922
8,交银新成长,519736.OF,王崇,M,20141022,交银施罗德基金,1.5,0.25,20140509,129.690903,20210930,-0.133661,0.471841,0.487781,0.033499,-0.224383
9,兴全趋势LOF,163402.SZ,童兰,F,20200702,兴证全球基金,1.5,0.25,20051103,333.099237,20210930,-0.064737,0.110949,0.140646,0.012392,-0.075863


In [10]:
def color_returns(val):
    if val >=0:
        color = '#EE7621' # light red 
    elif val <0:
        color = '#99ff66' # light green 
    else:
        color = '#FFFAFA' # ligth gray 
    return f'background-color: {color}'

format_dict = {'基金规模(亿元)': '￥{0:.1f}', 
               '管理费': '{0:.1f}', 
               '托管费': '{0:.2f}', 
               2017: '{0:.1%}', 
               2018: '{0:.1%}', 
               2019: '{0:.1%}', 
               2020: '{0:.1%}', 
               2021: '{0:.1%}', 
               2022: '{0:.1%}', 
                }


df.style.hide(axis='index')\
                    .hide(['规模对应日期','成立时间'],axis='columns')\
                    .format(format_dict)\
                    .applymap(color_returns,subset=year_list)\
                    .background_gradient(subset=['基金规模(亿元)'],cmap='Blues')

基金名称,基金代码,基金经理,性别,上任日期,基金公司,管理费,托管费,基金规模(亿元),2018,2019,2020,2021,2022
睿远成长价值A,007119.OF,傅鹏博,M,20190326,睿远基金,1.5,0.15,￥326.9,nan%,20.8%,71.0%,2.6%,-31.5%
中欧时代先锋A,001938.OF,刘伟伟,M,20211217,中欧基金,1.5,0.25,￥191.0,-8.3%,44.3%,37.5%,7.2%,-17.8%
景顺鼎益LOF,162605.SZ,刘彦春,M,20150710,景顺长城基金,1.5,0.25,￥226.5,-5.4%,19.6%,36.2%,-5.2%,-9.7%
广发双擎升级A,005911.OF,刘格菘,M,20181102,广发基金,1.5,0.25,￥141.1,-1.7%,121.7%,64.4%,4.2%,-27.5%
易方达蓝筹精选,005827.OF,张坤,M,20180905,易方达基金,1.5,0.25,￥698.5,-5.3%,55.1%,95.1%,-9.9%,-19.5%
易方达优质精选,110011.OF,张坤,M,20210910,易方达基金,1.5,0.25,￥216.6,-13.0%,58.6%,71.5%,-10.8%,-16.1%
富国天惠LOF,161005.SZ,朱少醒,M,20051116,富国基金,1.5,0.25,￥367.8,-11.3%,21.6%,25.7%,0.3%,-12.7%
华商未来主题,000800.OF,李双全,M,20190823,华商基金,1.5,0.25,￥5.1,-37.4%,40.3%,22.1%,16.0%,-23.6%
交银新成长,519736.OF,王崇,M,20141022,交银施罗德基金,1.5,0.25,￥129.7,-13.4%,47.2%,48.8%,3.3%,-22.4%
兴全趋势LOF,163402.SZ,童兰,F,20200702,兴证全球基金,1.5,0.25,￥333.1,-6.5%,11.1%,14.1%,1.2%,-7.6%


In [11]:
df.style.hide(axis='index')\
                    .hide(['规模对应日期','成立时间',2018,2019,2020,2021,2022],axis='columns')\
                    .format(format_dict)\
                    .applymap(color_returns,subset=year_list)\
                    .background_gradient(subset=['基金规模(亿元)'],cmap='Blues')

基金名称,基金代码,基金经理,性别,上任日期,基金公司,管理费,托管费,基金规模(亿元)
睿远成长价值A,007119.OF,傅鹏博,M,20190326,睿远基金,1.5,0.15,￥326.9
中欧时代先锋A,001938.OF,刘伟伟,M,20211217,中欧基金,1.5,0.25,￥191.0
景顺鼎益LOF,162605.SZ,刘彦春,M,20150710,景顺长城基金,1.5,0.25,￥226.5
广发双擎升级A,005911.OF,刘格菘,M,20181102,广发基金,1.5,0.25,￥141.1
易方达蓝筹精选,005827.OF,张坤,M,20180905,易方达基金,1.5,0.25,￥698.5
易方达优质精选,110011.OF,张坤,M,20210910,易方达基金,1.5,0.25,￥216.6
富国天惠LOF,161005.SZ,朱少醒,M,20051116,富国基金,1.5,0.25,￥367.8
华商未来主题,000800.OF,李双全,M,20190823,华商基金,1.5,0.25,￥5.1
交银新成长,519736.OF,王崇,M,20141022,交银施罗德基金,1.5,0.25,￥129.7
兴全趋势LOF,163402.SZ,童兰,F,20200702,兴证全球基金,1.5,0.25,￥333.1


In [13]:
df.style.hide(axis='index')\
                    .hide(['规模对应日期','成立时间','基金代码','性别','上任日期','基金公司','管理费','托管费','基金规模(亿元)'],axis='columns')\
                    .format(format_dict)\
                    .applymap(color_returns,subset=year_list)

基金名称,基金经理,2018,2019,2020,2021,2022
睿远成长价值A,傅鹏博,nan%,20.8%,71.0%,2.6%,-31.5%
中欧时代先锋A,刘伟伟,-8.3%,44.3%,37.5%,7.2%,-17.8%
景顺鼎益LOF,刘彦春,-5.4%,19.6%,36.2%,-5.2%,-9.7%
广发双擎升级A,刘格菘,-1.7%,121.7%,64.4%,4.2%,-27.5%
易方达蓝筹精选,张坤,-5.3%,55.1%,95.1%,-9.9%,-19.5%
易方达优质精选,张坤,-13.0%,58.6%,71.5%,-10.8%,-16.1%
富国天惠LOF,朱少醒,-11.3%,21.6%,25.7%,0.3%,-12.7%
华商未来主题,李双全,-37.4%,40.3%,22.1%,16.0%,-23.6%
交银新成长,王崇,-13.4%,47.2%,48.8%,3.3%,-22.4%
兴全趋势LOF,童兰,-6.5%,11.1%,14.1%,1.2%,-7.6%



## 相关阅读

- [视频：Plotly 和 Dash 在投资领域的应用](https://mp.weixin.qq.com/s/VopPrpe_64j5za4AYkmUTg)

- [视频：Plotly中绘制股票交易图表](https://mp.weixin.qq.com/s/1TUB2G-xavNm796uXUnRfQ)

- [Plotly中绘制三种经典的股票交易图表](https://mp.weixin.qq.com/s/1v3h5-5fNVk7cBdaRKzJkg)

- [神器Tushare，财经数据必备工具！](https://mp.weixin.qq.com/s/c1ukemeK12flCgA-lo69fA)

- [股票亏惨了，用Python来做一个投资计划](https://mp.weixin.qq.com/s/WYuMwCJBrWaBiDs8xp2KMA)

- [用 Python 读取巴菲特近期持仓数据](https://mp.weixin.qq.com/s/6h9Xq2lfrZSlRNw_W8aWcg)

更多内容，请关注公众号「**Python数据之道**」：

[![](https://tva1.sinaimg.cn/large/e6c9d24egy1h0iiejjflzj20go05kdhg.jpg)](https://mp.weixin.qq.com/mp/appmsgalbum?__biz=MzI2NjY5NzI0NA==&action=getalbum&album_id=2293754972943122444#wechat_redirect)


[![](https://tva1.sinaimg.cn/large/e6c9d24egy1h0iieh8vk4j20go05kaag.jpg)](https://mp.weixin.qq.com/mp/appmsgalbum?__biz=MzI2NjY5NzI0NA==&action=getalbum&album_id=1370549534602133504&scene=21#wechat_redirect)


![](https://tva1.sinaimg.cn/large/008i3skNgy1gs2d9ca7dvj31hc0u07ng.jpg)